## Modelo de lenguaje con tokenización por caracteres

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


**Importamos las librerias necesarias**

In [ ]:
pip install tensorflow

In [12]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from sklearn.datasets import fetch_20newsgroups

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

**Cargamos el conjunto de datos**

In [4]:
categories = ['sci.space', 'comp.graphics', 'rec.sport.hockey']
newsgroups_data = fetch_20newsgroups(subset='all', categories=categories)
corpus = newsgroups_data.data

**Tokenización**

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

**Padding para asegurar que todas las secuencias tengan el mismo largo**

In [6]:
maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)

**Dividimos en entrenamiento y validación**

In [7]:
X_train, X_val = train_test_split(data, test_size=0.2, random_state=42)

**Estructuración del dataset**

In [8]:
X_train_sequences = X_train[:, :-1]
y_train = X_train[:, -1]

X_val_sequences = X_val[:, :-1]
y_val = X_val[:, -1]

**Arquitectura**

In [10]:
# Creamos el modelo
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen-1))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


**Entrenamiento del modelo**

In [11]:
model.fit(X_train_sequences, y_train, epochs=10, validation_data=(X_val_sequences, y_val), batch_size=64)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 18s 400ms/step - accuracy: 0.0606 - loss: 10.4403 - val_accuracy: 0.0794 - val_loss: 9.0152
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 425ms/step - accuracy: 0.0868 - loss: 8.4972 - val_accuracy: 0.0794 - val_loss: 8.1666
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 406ms/step - accuracy: 0.0816 - loss: 7.7069 - val_accuracy: 0.0794 - val_loss: 7.8904
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 385ms/step - accuracy: 0.0851 - loss: 7.2775 - val_accuracy: 0.0794 - val_loss: 7.7740
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 21s 390ms/step - accuracy: 0.0778 - loss: 7.0253 - val_accuracy: 0.0794 - val_loss: 7.7169
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 393ms/step - accuracy: 0.0828 - loss: 6.8333 - val_accuracy: 0.0794 - val_loss: 7.6889
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 22s 425ms/step - accuracy: 0.0853 - loss: 6.7728 - val_accuracy: 0.0794 - val_loss: 7.6677
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 19s 379ms/step - accuracy: 0.0867 - loss: 6.7014 - val_acc

**Comentario**

El modelo de lenguaje entrenado mostró una pérdida en descenso (de 10.44 a 6.63) a lo largo de 10 épocas, pero la precisión se mantuvo baja (aproximadamente 8%) tanto en el conjunto de entrenamiento como en el de validación. Esto sugiere que el modelo no logró aprender de manera efectiva los patrones contextuales en el corpus. Es probable que el modelo sea demasiado simple o que se requieran ajustes en los hiperparámetros y el preprocesamiento.

**Mejoras implementadas**

* Más capas LSTM: Ahora tienes una red más profunda, con dos capas LSTM.
* Dropout: Se añadió Dropout para prevenir sobreajuste.
* Batch Normalization: Estabiliza y acelera el proceso de aprendizaje.
* Mayor dimensión de embeddings: Se aumentó a 256 para capturar mejor las relaciones entre palabras.
*Optimizador Adam: Mejora la velocidad de convergencia y rendimiento.

In [13]:
# Definimos los parámetros
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 256  # Aumentamos la dimensión del embedding
maxlen = 100

# Creamos el modelo
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen-1))

# Añadimos una capa LSTM adicional con return_sequences=True
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))  # Añadimos Dropout para evitar el sobreajuste

# Segunda capa LSTM para profundizar el modelo
model.add(LSTM(128))
model.add(Dropout(0.2))

# Añadimos Batch Normalization para estabilizar el entrenamiento
model.add(BatchNormalization())

# Capa de salida con activación softmax
model.add(Dense(vocab_size, activation='softmax'))

# Compilamos el modelo con el optimizador Adam
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenamos el modelo
model.fit(X_train_sequences, y_train, epochs=10, validation_data=(X_val_sequences, y_val), batch_size=64)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


37/37 ━━━━━━━━━━━━━━━━━━━━ 36s 816ms/step - accuracy: 0.0788 - loss: 10.5063 - val_accuracy: 0.1639 - val_loss: 10.4400
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 40s 799ms/step - accuracy: 0.1744 - loss: 9.3215 - val_accuracy: 0.1892 - val_loss: 9.8730
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 40s 777ms/step - accuracy: 0.2437 - loss: 7.1632 - val_accuracy: 0.2044 - val_loss: 8.8916
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 41s 785ms/step - accuracy: 0.2740 - loss: 5.2205 - val_accuracy: 0.2399 - val_loss: 7.7500
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 31s 829ms/step - accuracy: 0.3571 - loss: 3.9533 - val_accuracy: 0.2703 - val_loss: 7.1390
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 40s 792ms/step - accuracy: 0.4305 - loss: 3.3637 - val_accuracy: 0.2821 - val_loss: 6.8186
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 40s 774ms/step - accuracy: 0.5068 - loss: 2.8087 - val_accuracy: 0.3041 - val_loss: 6.7057
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 41s 778ms/step - accuracy: 0.5774 - loss: 2.3808 - val_accuracy: 0.3

**Comentario**

El modelo mejoró notablemente en comparación con el modelo anterior. La precisión en el entrenamiento aumentó de 8% a 69.9% y en la validación de 7.9% a 34.3%. La pérdida de entrenamiento se redujo de 10.44 a 1.70, indicando un mejor ajuste del modelo. Aunque la pérdida de validación sigue alta (7.07), muestra una tendencia a mejorar. En general, las modificaciones realizadas han tenido un impacto positivo en el rendimiento.

**Generación de secuencias**

**Greedy Search:**

In [14]:
def greedy_search(seed_text, model, tokenizer, max_len):
    for _ in range(max_len):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=maxlen-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = tokenizer.index_word[predicted[0]]
        seed_text += " " + output_word
    return seed_text

**probamos**

In [16]:
seed_text = "El clima en"
max_len = 20
generated_text = greedy_search(seed_text, model, tokenizer, max_len)
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
El clima en mark mark mark mark 01wb mail mail mail com edu edu mail edu edu edu au au champions champions champions


**Efecto de la Temperatura en Stochastic Beam Search**

In [17]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_with_temperature(seed_text, model, tokenizer, max_len, temperature=1.0):
    for _ in range(max_len):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=maxlen-1, padding='pre')
        predictions = model.predict(token_list, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_word = tokenizer.index_word[next_index]
        seed_text += " " + next_word
    return seed_text

**Probamos**

In [19]:
seed_text = "La inteligencia artificial"
max_len = 20
temperature = 0.7

In [20]:
generated_text = generate_with_temperature(seed_text, model, tokenizer, max_len, temperature)
print("Texto Generado:")
print(generated_text)

Texto Generado:
La inteligencia artificial mark mark mark obligations mail 01wb 6507 01wb mark daemon thanks thanks ca edu edu 1980 edu edu canada edu
